In [1]:
from torch.utils.data import DataLoader
from torch import optim
from torch.backends import cudnn
import numpy as np
import time
import csv
import warnings

from lib.aperture_dataset import ApertureDataset
from lib.lenet import LeNet
from lib.fit import fit

log_fname = 'results_%s.csv' % time.strftime('%Y%m%d-%H%M%S')

k = 4
fname_train = 'training_data/training_data.h5'
fname_validate = 'training_data/validation_data.h5'
save_path = 'save_here'


num_samples = 10 ** 5
dat_train = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_train2 = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_validate = ApertureDataset(fname_validate, num_samples, k)


/home/chenz28/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# TODO: Implement Dropout

using_cuda = True
if not using_cuda: warnings.warn("Not using CUDA")
# if using_cuda: cudnn.benchmark = True # CHANGED

num_threads = 16
if num_threads is None: num_threads = 4

# optimization parameters
# try_learning_rates = [1.0, 0.75, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001]
learning_rate = 0.05
optimization_algo = "Momentum"
if optimization_algo == "Momentum": momentum = 0.9
# try_batch_sizes = [32, 64, 128]
batch_size = 32
# NOTE we are using early stopping where we stop training once validation cost stops decreasing.
train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True) # CHANGED
train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True) # CHANGED
validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True) # CHANGED

# Model parameters
input_size = 65
output_size = 130

# conv1
try_conv1_kernel_sizess = list(range(2, 15))
try_conv1_num_kernels = list(range(2, 33))
try_conv1_strides = [1, 2, 3]

# pool1
try_pool1_sizes = [2, 3]

# conv2
# TODO in loop: conv2_kernel_size should <= conv1_kernel_size
# TODO in loop: conv2_num_kernels should >= conv1_num_kernels
try_conv2_kernel_sizess = list(range(2, 10))
try_conv2_num_kernels = list(range(2, 33))
try_conv2_strides = [1, 2]

# pool2
try_pool2_sizes = [2, 3]

# fcs
try_fc_hidden_sizes = [16, 32, 64, 128, 256, 512] # CHANGED: eliminated 8, 1024
try_fc_num_hidden_layers = [1, 2, 3]
                    

# Random Hyperparameter Search
def choose_int(array): return int(np.random.choice(array)) # int() because PyTorch doesn't convert np.int64 to int.

num_trys = 100
run_count = 0

print()
print("| #  | vali cost | diff% | learn_rate | batch_size | conv1_kernel_size | conv1_num_kernels | conv1_stride | pool1_kernels | conv2_kernel_size | conv2_kernels | conv2_strides | pool2_kernels | fcs_size | fcs_num_hidden_layers |")
print("| -- | --------- | ----- | ---------- | ---------- | ----------------- | ----------------- | ------------ | ------------- | ----------------- | ------------- | ------------- | ------------- | -------- | --------------------- |")

while run_count < num_trys:
    # choose random hyperparameters: optimization
#     batch_size = choose_int(try_batch_sizes)
#     learning_rate = np.random.choice(try_learning_rates)
    
    # choose random hyperparameters: model
    conv1_kernel_size = choose_int(try_conv1_kernel_sizess)
    conv1_num_kernels = choose_int(try_conv1_num_kernels)
    conv1_stride = choose_int(try_conv1_strides)
    
    # enforce relative shape and divisibility
    conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
    while isinstance(conv1_output_size[1], float) and not conv1_output_size[1].is_integer():
        # conv1_output_size must be an integer
        # random search is not the most efficient approach but I'm too lazy to filter right now.
        conv1_stride = choose_int(try_conv1_strides)
        conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
        
    
    pool1_kernel_size = choose_int(try_pool1_sizes)
    pool1_stride = 2
    pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
    while isinstance(pool1_output_size[1], float) and not pool1_output_size[1].is_integer():
        # conv1_output_size must be an integer
        pool1_kernel_size = choose_int(try_pool1_sizes)
        pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
    
    conv2_kernel_size = choose_int(try_conv2_kernel_sizess)
    conv2_num_kernels = choose_int(try_conv2_num_kernels)
    conv2_stride = choose_int(try_conv2_strides)
    
    conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)
    while isinstance(conv2_output_size[1], float) and not conv2_output_size[1].is_integer():
        # conv2_output_size must be an integer
        conv2_stride = choose_int(try_conv2_strides)
        conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)

    pool2_kernel_size = choose_int(try_pool2_sizes)
    pool2_stride = 2
    pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
    
    while isinstance(pool2_output_size[1], float) and not pool2_output_size[1].is_integer():
        # conv1_output_size must be an integer
        pool2_kernel_size = choose_int(try_pool2_sizes)
        pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
    
    fcs_hidden_size = choose_int(try_fc_hidden_sizes)
    fcs_num_hidden_layers = choose_int(try_fc_num_hidden_layers)
    
    # load data because of difference batch_sizes
#     train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=False) # CHANGED
#     train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=False) # CHANGED
#     validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=False) # CHANGED
    
    try:
        # build the model and train
        model = LeNet(input_size, output_size, fcs_hidden_size, fcs_num_hidden_layers,
                      batch_size,
                      pool1_kernel_size,
                      conv1_kernel_size, conv1_num_kernels, conv1_stride,
                      pool2_kernel_size,
                      conv2_kernel_size, conv2_num_kernels, conv2_stride)

        if using_cuda: model.cuda()
        model.share_memory() # gradients are allocated lazily, so they are not shared here
        optimizer = optim.SGD(model.parameters(), learning_rate, momentum=momentum)

        cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)


        print("| %02d | %.7f | %0.3f | %0.7f  | %d        | %02d                | %02d                 | %d            | %d             | %d                | %d             | %d             | %d           | %04d     | %s |" % \
              (run_count,cost, diff,learning_rate,batch_size,conv1_kernel_size,  conv1_num_kernels,      conv1_stride,   pool1_kernel_size,    conv2_kernel_size,  conv2_num_kernels,   conv2_stride, pool2_kernel_size,    fcs_hidden_size, fcs_num_hidden_layers))
        run_count += 1
    except:
        print("| %02d | error | error | %0.7f  | %d        | %02d                | %02d                 | %d            | %d             | %d                | %d             | %d             | %d           | %04d     | %s |" % \
              (run_count,learning_rate,batch_size,conv1_kernel_size,  conv1_num_kernels,      conv1_stride,   pool1_kernel_size,    conv2_kernel_size,  conv2_num_kernels,   conv2_stride, pool2_kernel_size,    fcs_hidden_size, fcs_num_hidden_layers))
        

print("\nIt's all over.")


| #  | vali cost | diff% | learn_rate | batch_size | conv1_kernel_size | conv1_num_kernels | conv1_stride | pool1_kernels | conv2_kernel_size | conv2_kernels | conv2_strides | pool2_kernels | fcs_size | fcs_num_hidden_layers |
| -- | --------- | ----- | ---------- | ---------- | ----------------- | ----------------- | ------------ | ------------- | ----------------- | ------------- | ------------- | ------------- | -------- | --------------------- |


/home/chenz28/beam_nn/lib/lenet.py:53: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.conv1.weight.data)
/home/chenz28/beam_nn/lib/lenet.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.conv2.weight.data)
/home/chenz28/beam_nn/lib/fully_connected_net.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(layer.weight.data)


E: 01 / Train: 2.263e-02 / Valid: 2.403e-02 / Diff Valid: -99.98% / Diff Valid-Train: 6.2% / Time: 12.21
E: 02 / Train: 1.110e-02 / Valid: 1.201e-02 / Diff Valid: -50.04% / Diff Valid-Train: 8.1% / Time: 11.15
E: 03 / Train: 6.543e-03 / Valid: 7.173e-03 / Diff Valid: -40.25% / Diff Valid-Train: 9.6% / Time: 12.00
E: 04 / Train: 4.971e-03 / Valid: 5.479e-03 / Diff Valid: -23.61% / Diff Valid-Train: 10.2% / Time: 11.62
E: 05 / Train: 3.680e-03 / Valid: 4.078e-03 / Diff Valid: -25.57% / Diff Valid-Train: 10.8% / Time: 12.11
E: 06 / Train: 2.422e-03 / Valid: 2.677e-03 / Diff Valid: -34.35% / Diff Valid-Train: 10.5% / Time: 11.81
E: 07 / Train: 1.490e-03 / Valid: 1.647e-03 / Diff Valid: -38.48% / Diff Valid-Train: 10.5% / Time: 11.56
E: 08 / Train: 1.106e-03 / Valid: 1.224e-03 / Diff Valid: -25.69% / Diff Valid-Train: 10.6% / Time: 10.71
E: 09 / Train: 9.751e-04 / Valid: 1.077e-03 / Diff Valid: -12.04% / Diff Valid-Train: 10.4% / Time: 10.75
E: 10 / Train: 9.362e-04 / Valid: 1.034e-03 / Dif